In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split, SequentialSampler
from tqdm import tqdm
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [2]:
df =  pd.read_csv("final_df.csv")

In [3]:
df.drop(['Unnamed: 0'], axis=1,  inplace=True)

In [4]:
df.head()

,text,sadness,joy,love,anger,fear,surprise,total
0,That game hurt.,1,0,0,0,0,0,1
1,Man I love reddit.,0,0,1,0,0,0,1
2,So happy for [NAME]. So sad he's not here. Ima...,1,1,0,0,0,0,2
3,"I just came home, what the fuck is this lineup...",0,0,1,0,0,0,1
4,By far the coolest thing I've seen on this thr...,0,1,0,0,0,0,1


In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['text']).toarray()
X_test_tfidf = tfidf_vectorizer.transform(test_df['text']).toarray()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

In [ ]:
def prepare_data_for_bert(texts, labels, tokenizer):
    
    encoded_data = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    labels = torch.tensor(labels)
    return TensorDataset(encoded_data['input_ids'], encoded_data['attention_mask'], labels)

train_data = prepare_data_for_bert(train_df['text'], train_df[['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']].values, tokenizer)
test_data = prepare_data_for_bert(test_df['text'], test_df[['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']].values, tokenizer)

In [26]:
df['text'] = df['text'].astype("string")
label_columns = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
df[label_columns] = df[label_columns].astype("float")

In [30]:
train_data = prepare_data_for_bert(texts=train_texts, labels=train_labels, tokenizer=tokenizer)
test_data = prepare_data_for_bert(texts=test_texts, labels=test_labels, tokenizer=tokenizer)

NameError: name 'train_texts' is not defined

In [29]:
# Ensure 'text' column is in string format
df['text'] = df['text'].astype(str)

# Prepare data for BERT
texts = df['text'].tolist()  # Convert 'text' column to a list of strings
labels = df[['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']].values.astype(float)

# Now, 'texts' is a list of strings and 'labels' is a numpy array of floats

In [27]:
df.dtypes

text        string[python]
sadness            float64
joy                float64
love               float64
anger              float64
fear               float64
surprise           float64
total              float64
dtype: object

In [ ]:
# Ensure 'text' column is in string format
df['text'] = df['text'].astype(str)

# Prepare data for BERT
texts = df['text'].tolist()  # Convert 'text' column to a list of strings
labels = df[['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']].values.astype(float)

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Now, 'train_texts', 'test_texts' are lists of strings, and 'train_labels', 'test_labels' are numpy arrays of floats

train_data = prepare_data_for_bert(texts=train_texts, labels=train_labels, tokenizer=tokenizer)
test_data = prepare_data_for_bert(texts=test_texts, labels=test_labels, tokenizer=tokenizer)